In [12]:
import numpy as np
import os
import time
import datetime
import threading
import pandas as pd
import pickle
from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose
import matplotlib.pyplot as plt
import cv2
import mediapipe as mp
from typing import Mapping
mp_pose = mp.solutions.pose

from audSample import *
from createAudio import *

In [13]:
classifier = pickle.load(open('yoga_model/latest_model1', 'rb'))

In [14]:
# to calculate angle in three points
def getAngles(item, pt1, pt2, pt3):
    a = item[pt1]
    b = item[pt2]
    c = item[pt3]
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = (np.arccos(cosine_angle))*(180/np.pi)
    return angle

In [15]:
def chk_vis(lis):
    cnt = 0
    # lis[11] right shoulder, lis[12] left shoulder, lis[23] right hip, lis[24] left hip, lis[27], lis[28]
    if (
        (lis[11].visibility>0.5 or lis[12].visibility>0.5) and
        (lis[23].visibility>0.5 or lis[24].visibility>0.5) and
        (lis[27].visibility>0.5 or lis[28].visibility>0.5)
        ):  return True
    else: return False 
# def chk_vis(lis):
#     cnt = 0
#     for obj in lis:
#         if (obj.visibility>0.5): cnt+=1
#     if (cnt==33): return True
#     else: return False 

In [16]:

from timeit import default_timer as timer

PosesAudioLoc = "audioFiles/poses/"
SecAudioLoc = "audioFiles/seconds/"


In [17]:
import json
colr = 255
with open("pose_angle.json", "r") as pose_angle:
    pose_data = json.load(pose_angle)
def chck_angle(ps_name, angle_dic):
    angle_col_dic = {}
    for d_angle in pose_data[ps_name]:
        mn, std = pose_data[ps_name][d_angle]
        rat = abs(mn-angle_dic[d_angle])/mn
        if rat>1: rat = 1
        angle_col_dic[d_angle] = (0, colr*(1-rat), colr*rat)
    # print(angle_col_dic)
    return angle_col_dic
# print(pose_data)    

In [18]:
idealLoc = f"images/step"
ideal_poses = {
    "pranamasan":f"{idealLoc}1.jpg",
    "urdhva hastasana":f"{idealLoc}2.jpg",
    "uttanasana":f"{idealLoc}3.jpg",
    "aswa sanchalnasan":f"{idealLoc}4.jpg",
    "adho mukha savanasana":f"{idealLoc}5.jpg",
    "ashtanga namaskar":f"{idealLoc}6.jpg",
    "bhujangasana":f"{idealLoc}7.jpg"
}

In [19]:
recently = []
steady_time = []
def start_vid(frame_rate):
    secCheck = 0
    vid = cv2.VideoCapture(0)
    # frame_rate = 30
    prev = time.time()
    threadChk = 0
    while(True):
        
        # Capture the video frame
        # by frame
        time_elapsed = time.time() - prev
        ret, frame = vid.read()
        # time_elapsed = time.time() - prev
        # ret, frame = self.video.read()
        if time_elapsed > 1./frame_rate:
            prev = time.time()
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            with mp_pose.Pose() as pose_tracker:
                result = pose_tracker.process(image=frame)
                # mp_drawing.plot_landmarks(results.pose_world_landmarks, mp_pose.POSE_CONNECTIONS)
                pose_landmarks = result.pose_landmarks
            output_frame = frame.copy()
            # print(pose_landmarks.landmark)
            pose_name = ""
            if pose_landmarks is not None:
                dict_obj = {}
                for i in range(0,33):
                    dict_obj[i] = mp_drawing.DrawingSpec(color=mp_drawing.WHITE_COLOR)
                pose_landmarka = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
                output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
                frame_height, frame_width = frame.shape[:2]
                pose_landmarka *= np.array([frame_width, frame_height, frame_width])
                # uncomment this when predicting
                item = pose_landmarka
                angle_dict = {
                    'left_shoulder': getAngles(item, 13, 11, 12),
                    'right_shoulder': getAngles(item, 24, 12, 14),
                    'left_elbow': getAngles(item, 15, 13, 11),
                    'right_elbow': getAngles(item, 12, 14, 16),
                    'left_leg_waist': getAngles(item, 11, 23, 25),
                    'right_leg_waist': getAngles(item, 12, 24, 26),
                    'left_knee': getAngles(item, 23, 25, 27),
                    'right_knee': getAngles(item, 24, 26, 28),
                }
                arr = np.array([angle_dict['left_shoulder'],
                angle_dict['right_shoulder'],
                angle_dict['left_elbow'],
                angle_dict['right_elbow'],
                angle_dict['left_leg_waist'],
                angle_dict['right_leg_waist'],
                angle_dict['left_knee'],
                angle_dict['right_knee']])
                if chk_vis(pose_landmarks.landmark):
                    res = classifier.predict([arr]) # classified pose name
                    pose_name = res[0]
                    angle_colr = chck_angle(pose_name, angle_dict)
                    dict_obj[11] = mp_drawing.DrawingSpec(color=angle_colr["left_shoulder"])
                    dict_obj[12] = mp_drawing.DrawingSpec(color=angle_colr["right_shoulder"])
                    dict_obj[13] = mp_drawing.DrawingSpec(color=angle_colr["left_elbow"])
                    dict_obj[14] = mp_drawing.DrawingSpec(color=angle_colr["right_elbow"])
                    dict_obj[23] = mp_drawing.DrawingSpec(color=angle_colr["left_leg_waist"])
                    dict_obj[24] = mp_drawing.DrawingSpec(color=angle_colr["right_leg_waist"])
                    dict_obj[25] = mp_drawing.DrawingSpec(color=angle_colr["left_knee"])
                    dict_obj[26] = mp_drawing.DrawingSpec(color=angle_colr["right_knee"])
                    x: Mapping[int, mp_drawing.DrawingSpec] = dict_obj
                    mp_drawing.draw_landmarks(
                        image=output_frame,
                        landmark_list=pose_landmarks,
                        connections=mp_pose.POSE_CONNECTIONS,
                        landmark_drawing_spec=  x)
                assert len(pose_landmarks.landmark) == 33, 'Unexpected number of predicted pose landmarks: {}'.format(len(pose_landmarks.landmark))
                # for i in pose_landmarks.landmark:
                #     print(i)
            cv2.rectangle(output_frame, (0,0), (650, 50), (0,0,0), -1)
            output_frame = cv2.putText(output_frame, pose_name, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2, cv2.LINE_AA)
            # cv2.imshow('frame', output_frame)
            c_time = timer()
            
            # ideal=1
            # for i in angle_colr.values():
            #     if i[2] > 150: ideal = 0  # if red detected clear the timer

            if pose_name!="":
                image = cv2.imread(ideal_poses[pose_name])
                image = cv2. resize(image, (550,470))
                cv2.imshow("ideal", image)

            if pose_name != "":
                
                if 1:
                    steady_time_now = timer()
                    steadyF = 0
                    if not steady_time:
                        steady_time.append(steady_time_now)
                    elif (steady_time_now - steady_time[0])>3:
                        steadyF = 1
                    else:
                        # print(steady_time_now-steady_time[0])
                        steady_time.append(steady_time_now)
                        
                    output_frame = cv2.putText(output_frame, str("{:.0f}".format(steady_time_now-steady_time[0]))+" s", (300,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                    if steadyF == 1:
                        countSay = "{:.0f}".format(steady_time_now-steady_time[0])
                        print(angle_colr)
                        if not recently: # classify and say
                            t1 = threading.Thread(target=start_playlist, args=[[ f"{PosesAudioLoc}{pose_name}.wav"]])
                            t1.start()
                            threadChk = 1
                            recently.append(c_time)

                        elif secCheck==0:
                            if int(countSay)%15 == 0:
                                print(countSay)
                                t1 = threading.Thread(target=start_playlist, args=[[ f"{SecAudioLoc}{countSay}sec.wav"]])
                                t1.start()
                                recently.pop()
                                recently.append(c_time)
                                secCheck = 1
                            else:
                                secCheck = 0
                        
                        if secCheck==1 and int(countSay)%15 != 0: secCheck=0
                    
                    
            else:
                steady_time.clear()
                recently.clear()

            cv2.imshow('frame', output_frame)
        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break


    if threadChk: t1.join()
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()

In [20]:
classifier.classes_

array(['adho mukha savanasana', 'ashtanga namaskar', 'aswa sanchalnasan',
       'bhujangasana', 'pranamasan', 'urdhva hastasana', 'uttanasana'],
      dtype=object)

In [21]:
# classifier.metric_params()

In [22]:
import warnings
warnings.filterwarnings('ignore')
start_vid(10)

Exception in thread Thread-9:
Traceback (most recent call last):
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\sem 7\mini-project\project\YogaPoseDetect\audSample.py", line 26, in start_playlist
    pygame.mixer.music.queue(playList[0])
IndexError: list index out of range


{'left_shoulder': (0, 224.6165716017707, 30.383428398229317), 'right_shoulder': (0, 110.8753338357159, 144.1246661642841), 'left_elbow': (0, 210.99831116467496, 44.00168883532503), 'right_elbow': (0, 200.5843087425071, 54.41569125749292), 'left_leg_waist': (0, 166.17631082992523, 88.82368917007477), 'right_leg_waist': (0, 131.9260868557614, 123.0739131442386), 'left_knee': (0, 242.5850724447743, 12.41492755522569), 'right_knee': (0, 81.3665699832928, 173.6334300167072)}
{'left_shoulder': (0, 232.0775529938176, 22.9224470061824), 'right_shoulder': (0, 149.79328246452155, 105.20671753547848), 'left_elbow': (0, 211.6501755622686, 43.3498244377314), 'right_elbow': (0, 216.08325069200086, 38.916749307999154), 'left_leg_waist': (0, 209.0087061395831, 45.99129386041691), 'right_leg_waist': (0, 112.43231938841392, 142.5676806115861), 'left_knee': (0, 240.5533346525628, 14.446665347437207), 'right_knee': (0, 130.45305010709833, 124.5469498929017)}
{'left_shoulder': (0, 234.99828866058675, 20.00

Exception in thread Thread-10:
Traceback (most recent call last):
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\sem 7\mini-project\project\YogaPoseDetect\audSample.py", line 26, in start_playlist
    pygame.mixer.music.queue(playList[0])
IndexError: list index out of range


{'left_shoulder': (0, 242.67796021645268, 12.322039783547346), 'right_shoulder': (0, 100.2178973482824, 154.7821026517176), 'left_elbow': (0, 214.30324383687355, 40.69675616312646), 'right_elbow': (0, 206.4489499098616, 48.55105009013839), 'left_leg_waist': (0, 240.48713047071567, 14.512869529284325), 'right_leg_waist': (0, 119.5400933776773, 135.4599066223227), 'left_knee': (0, 232.65171622320844, 22.34828377679158), 'right_knee': (0, 88.8448071814595, 166.15519281854048)}
15
{'left_shoulder': (0, 235.79389767941615, 19.206102320583852), 'right_shoulder': (0, 100.28484007847325, 154.71515992152675), 'left_elbow': (0, 210.26656884782437, 44.73343115217561), 'right_elbow': (0, 217.702765876118, 37.297234123882), 'left_leg_waist': (0, 237.31004897194487, 17.68995102805514), 'right_leg_waist': (0, 138.30893365326602, 116.69106634673398), 'left_knee': (0, 246.11672645529364, 8.883273544706338), 'right_knee': (0, 122.93724453168035, 132.06275546831966)}
{'left_shoulder': (0, 240.76468505384

Exception in thread Thread-11:
Traceback (most recent call last):
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\sem 7\mini-project\project\YogaPoseDetect\audSample.py", line 26, in start_playlist
    pygame.mixer.music.queue(playList[0])
IndexError: list index out of range


...

{'left_shoulder': (0, 178.88956520865003, 76.11043479135), 'right_shoulder': (0, 241.6322921764859, 13.367707823514122), 'left_elbow': (0, 220.38787820812146, 34.61212179187856), 'right_elbow': (0, 210.78146010572254, 44.21853989427744), 'left_leg_waist': (0, 160.01041378618137, 94.98958621381864), 'right_leg_waist': (0, 217.06969670045734, 37.93030329954268), 'left_knee': (0, 215.09890801659034, 39.901091983409636), 'right_knee': (0, 244.80816207723421, 10.191837922765798)}
{'left_shoulder': (0, 183.96244546599274, 71.03755453400726), 'right_shoulder': (0, 234.44036753187913, 20.559632468120864), 'left_elbow': (0, 213.46685548737304, 41.533144512626954), 'right_elbow': (0, 215.5102987269798, 39.48970127302019), 'left_leg_waist': (0, 62.2078210090049, 192.7921789909951), 'right_leg_waist': (0, 72.45792785691566, 182.54207214308434), 'left_knee': (0, 78.0800137854688, 176.9199862145312), 'right_knee': (0, 153.20440003369953, 101.79559996630047)}
{'left_shoulder': (0, 182.70766433648035,

Exception in thread Thread-16:
Traceback (most recent call last):
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\sem 7\mini-project\project\YogaPoseDetect\audSample.py", line 26, in start_playlist
    pygame.mixer.music.queue(playList[0])
IndexError: list index out of range


{'left_shoulder': (0, 221.81498833470454, 33.185011665295455), 'right_shoulder': (0, 182.3907302085526, 72.60926979144737), 'left_elbow': (0, 111.43969799286167, 143.56030200713835), 'right_elbow': (0, 231.17457060314908, 23.825429396850943), 'left_leg_waist': (0, 68.75115599555266, 186.24884400444734), 'right_leg_waist': (0, 95.58176495180821, 159.41823504819178), 'left_knee': (0, 225.8235457416983, 29.17645425830169), 'right_knee': (0, 195.16445978817367, 59.835540211826334)}
{'left_shoulder': (0, 219.23565103993752, 35.76434896006246), 'right_shoulder': (0, 172.5568099101504, 82.4431900898496), 'left_elbow': (0, 114.802058943461, 140.197941056539), 'right_elbow': (0, 240.53917553770853, 14.460824462291482), 'left_leg_waist': (0, 63.75528618784318, 191.24471381215682), 'right_leg_waist': (0, 89.7738225884393, 165.2261774115607), 'left_knee': (0, 229.58821710828425, 25.41178289171576), 'right_knee': (0, 191.3507843637658, 63.649215636234196)}
{'left_shoulder': (0, 208.45556746392776, 

Exception in thread Thread-17:
Traceback (most recent call last):
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 932, in _bootstrap_inner
    self.run()
  File "c:\users\prathmesh waghmode\appdata\local\programs\python\python38\lib\threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "d:\sem 7\mini-project\project\YogaPoseDetect\audSample.py", line 26, in start_playlist
    pygame.mixer.music.queue(playList[0])
IndexError: list index out of range


{'left_shoulder': (0, 96.03372804395735, 158.96627195604265), 'right_shoulder': (0, 102.72600817071033, 152.27399182928966), 'left_elbow': (0, 149.94397410518584, 105.05602589481417), 'right_elbow': (0, 173.94987079881582, 81.05012920118419), 'left_leg_waist': (0, 173.8722798514688, 81.12772014853121), 'right_leg_waist': (0, 119.142711737369, 135.85728826263102), 'left_knee': (0, 184.27790540218072, 70.72209459781928), 'right_knee': (0, 186.9433337469282, 68.05666625307181)}
{'left_shoulder': (0, 106.7581702055493, 148.2418297944507), 'right_shoulder': (0, 84.18401401746246, 170.81598598253754), 'left_elbow': (0, 137.0653188106207, 117.9346811893793), 'right_elbow': (0, 197.98138264883156, 57.01861735116843), 'left_leg_waist': (0, 135.83125343670633, 119.16874656329364), 'right_leg_waist': (0, 83.01700051950495, 171.98299948049507), 'left_knee': (0, 185.44978962178615, 69.55021037821386), 'right_knee': (0, 191.17383074493736, 63.82616925506265)}
{'left_shoulder': (0, 94.7354201994926, 